### Question 2: Ridge Regression and LASSO
+ Generate dataset using `numpy`

In [1]:
import numpy as np
import RidgeRegression

In [6]:
m = 150
d = 75
X = np.random.rand(m, d)
theta = np.zeros(d)
indices_neg, indices_pos = np.split(np.random.choice(10, 10, replace=False), 2)
theta[indices_neg] = -10
theta[indices_pos] = 10

noise = np.random.normal(loc=0, scale=0.1, size=(m,))
y = np.dot(X, theta) + noise

print(theta.shape)
print(X.shape)
print(noise.shape)
print(y.shape)

(75,)
(150, 75)
(150,)
(150,)


In [ ]:
train_X, valid_X, test_X = X[:80], X[80:100], X[100:]
train_y, valid_y, test_y = y[:80], y[80:100], y[100:]